In [2]:
import cvxpy as cp
import numpy as np
import pandas as pd

"""
Own packages
"""

import codelib.portfolio_optimization.risk_metrics as rm
import codelib.portfolio_optimization.diversification as dm

from codelib.statistics.moments import corr_to_cov_matrix, cov_to_corr_matrix

from codelib.visualization.base import correlation_plot, fan_chart
from codelib.visualization.layout import DefaultStyle, default_colors
DefaultStyle();

from codelib.portfolio_optimization.mean_variance import portfolio_mean, portfolio_std, portfolio_variance

# Optimization with Conditional Value at Risk (CVaR)

## Convex Optimization

[Rockafeller and Uryasev (2000)](https://sites.math.washington.edu/~rtr/papers/rtr179-CVaR1.pdf) specified the optimization of CVaR as a convex optimization problem. Considering a portfolio with weights  $\boldsymbol{w}$  and $N$ vectors of simulated returns ($K \times 1$) $\boldsymbol{r}_i, i=1, ..., N$ , the CVaR at level  $1-\beta$  can be optimized by solving the following problem:

\begin{align*}
\min_{\boldsymbol{w}, \alpha, \boldsymbol{u
}} & \alpha + \frac{1}{N (1 - \beta)} \sum_{i=1}^{N} u_i \\
\text{s.t.} & \boldsymbol{w}^\top \boldsymbol{1} = 1 \\
& \boldsymbol{w} \geq 0 \\
& u_i \geq 0, \quad i=1, \ldots, N \\
& u_i \geq -\boldsymbol{w} ^\top \boldsymbol{r}_i - \alpha, \quad i=1, \ldots, N
\end{align*}    

In [3]:
"""
Define a function to optimize portfolio weights using CVaR minimization.
"""

def calculate_mean_cvar_optimization(returns: np.ndarray, beta: float, probs: None, return_target: float = None, verbose: bool = False) -> np.ndarray:

    """
    Optimize portfolio weights using CVaR minimization. The optimization problem can handle optional return constraints.

    Parameters
    ----------
    returns : np.ndarray
        Simulated returns of shape (num_simulations, num_assets).
    beta : float
        Confidence level for CVaR (e.g., 0.95 for 95% CVaR).
    probs : np.ndarray or None
        Probabilities associated with each simulation. If None, equal probabilities are assumed.
    return_target : float or None
        Target return for the portfolio. If None, no return constraint is applied.
    verbose : bool
        If True, print solver output.

    Returns
    -------
    w : np.ndarray
        Optimized portfolio weights.
    alpha : float
        Value at Risk at the specified confidence level.
    """

    num_assets = returns.shape[1]
    num_sim = returns.shape[0]

    if probs is None:
        probs = np.ones(num_sim) / num_sim

    # Define variables
    w = cp.Variable(num_assets)  # portfolio weights
    u = cp.Variable(num_sim)  # auxiliary variables for CVaR
    alpha = cp.Variable()  # Value at Risk variable

    # Define the objective function (minimize CVaR)
    objective = cp.Minimize(alpha + (1 / (1 - beta)) * cp.sum(cp.multiply(probs, u)))

    # Define constraints
    constraints = [
        cp.sum(w) == 1,  # weights sum to 1
        w >= 0,          # no short selling
        u >= 0,          # auxiliary variables non-negative
        u >= -returns @ w - alpha  # definition of u
    ]

    if return_target is not None:
        constraints.append(cp.sum(cp.multiply(probs, returns @ w)) >= return_target)

    # Solve the optimization problem
    prob = cp.Problem(objective, constraints)
    prob.solve(verbose=verbose)

    return w.value, alpha.value

In [4]:
"""
Simulate some return data for testing the CVaR optimization function.
"""

num_sim = 10_000

asset_names = ["Government bonds", "Investment-grade bonds", "High-yield bonds",
               "Emerging markets gov. bonds", "Equities (developed markets)",
               "Equities (Emerging markets)", "Private equity", "Infrastructure",
               "Real Estate", "Hedgefunds"]

corr_mat = np.array([[1.00, 0.60, 0.10, 0.30, -0.10, -0.10, -0.20, -0.10, -0.10, -0.10],
                     [0.60, 1.00, 0.60, 0.60, 0.20, 0.20, 0.20, 0.10, 0.10, 0.30],
                     [0.10, 0.60, 1.00, 0.70, 0.70, 0.60, 0.60, 0.40, 0.30, 0.70],
                     [0.30, 0.60, 0.70, 1.00, 0.50, 0.60, 0.40, 0.20, 0.20, 0.50],
                     [-0.10, 0.20, 0.70, 0.50, 1.00, 0.70, 0.80, 0.40, 0.40, 0.80],
                     [-0.10, 0.20, 0.60, 0.60, 0.70, 1.00, 0.70, 0.40, 0.40, 0.70],
                     [-0.20, 0.20, 0.60, 0.40, 0.80, 0.70, 1.00, 0.40, 0.40, 0.70],
                     [-0.10, 0.10, 0.40, 0.20, 0.40, 0.40, 0.40, 1.00, 0.30, 0.40],
                     [-0.10, 0.10, 0.30, 0.20, 0.40, 0.40, 0.40, 0.30, 1.00, 0.40],
                     [-0.10, 0.30, 0.70, 0.50, 0.80, 0.70, 0.70, 0.40, 0.40, 1.00]])

vols = np.array([3.7, 5.5, 11.9, 10.7, 15.3, 21.7, 20.4, 14.0, 10.8, 9.4]) / 100.0

cov_mat = corr_to_cov_matrix(corr_mat=corr_mat, vols=vols)

mu = np.array([1.9, 2.2, 4.9, 4.3, 6.1, 8.3, 10.2, 5.6, 4.1, 3.8]) / 100

sim_returns = np.random.multivariate_normal(mean=mu, cov=cov_mat, size=num_sim)


In [10]:
"""
Optimization
"""

w, alpha = calculate_mean_cvar_optimization(returns=sim_returns, beta=0.95,
                                            probs=None, return_target=0.05, verbose=False)

In [11]:
alpha

array(0.06618123)

In [16]:
w @ np.mean(sim_returns, axis=0)

np.float64(0.04999999999999387)

In [17]:
-rm.calculate_value_at_risk(sim_returns @ w, p=0.05)

np.float64(0.06618123458523924)

In [18]:
pd.DataFrame(w, index=asset_names, columns=["Weights"])

,Weights
Government bonds,4.999316e-01
Investment-grade bonds,7.529224e-14
High-yield bonds,5.632714e-14
Emerging markets gov. bonds,2.763154e-02
Equities (developed markets),2.606054e-14
Equities (Emerging markets),3.060784e-13
Private equity,2.827833e-01
Infrastructure,1.479001e-01
Real Estate,4.175347e-02
Hedgefunds,7.838771e-14
